In [27]:
import common
import tqdm
import numpy as np
import pandas as pd

In [28]:
! ls /data/zhenyus/webcachesim/trace

memcachier0app100m.tr	   synthetic_urm.tr		 w10m_modify.tr.ant
memcachier0app100m.tr.ant  synthetic_urm.tr.ant		 w10m.tr
memcachier100m.tr	   _t_memcachier0app100m.tr	 w1k.tr
memcachier100m.tr.ant	   _t_memcachier0app100m.tr.ant  w1k.tr.ant
msr_proj_1_20m.tr	   _t_memcachier100m.tr		 w1m.tr
msr_proj_1_20m.tr.ant	   _t_memcachier100m.tr.ant	 w1m.tr.ant
msr_proj_1.tr		   w100k.tr			 w200m.mapping
msr_proj_1.tr.ant	   w100k.tr.ant			 w200m.tr
msr_proj_2.tr		   w10k.tr			 w200m.tr.ant
msr_proj_2.tr.ant	   w10m_modify.tr		 w50mBHR.tr


In [19]:
# timing scaning file
%%timeit -n1 -r1
with open(f'{common.WEBCACHESIM_ROOT}/trace/w200m.tr.ant') as f:
    for line in f:
        fields = line.split()

1min 57s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [29]:
# check duplicate size
id_size = {}
with open(f'{common.WEBCACHESIM_ROOT}/trace/w200m.tr.ant') as f:
    for line in f:
        fields = line.split()
        t, idx, size, next_t = int(fields[0]), int(fields[1]), int(fields[2]), int(fields[3])
        if idx in id_size and id_size[idx] != size:
            print(1)
        id_size[idx] = size

In [3]:
max_n_past_intervals = 4
threshold = int(1e7)
log1p_threshold = np.log1p(threshold)
snapshot_interval = int(5e7)

In [4]:
class Meta:
    def __init__(self, key, size, past_timestamp, future_timestamp):
        self._key = key;
        self._size = size;
        self._past_timestamps = [past_timestamp];
        self._future_timestamp = future_timestamp;
    
    def update(self, past_timestamp, future_timestamp):
        self._past_timestamps.append(past_timestamp);
        if len(self._past_timestamps) > max_n_past_intervals:
            self._past_timestamps.pop(0)
        self._future_timestamp = future_timestamp

In [8]:
meta = {}  # key -> meta
snapshot = []

with open(f'{common.WEBCACHESIM_ROOT}/trace/w200m.tr.ant') as f:
    for line in f: 
        fields = line.split()
        t, key, size, next_t = int(fields[0]), int(fields[1]), int(fields[2]), int(fields[3])
        
        # snapshot
        if t % snapshot_interval == 0:
            s = [t//snapshot_interval]  # snapshot id
            for k, m in meta.items():
                if m._past_timestamps[-1] < t - threshold:
                    continue
                for i in range(-1, -max_n_past_intervals-1, -1):
                    if -i <= len(m._past_timestamps) and m._past_timestamps[i] >= t - threshold:
                        s.append(t - m._past_timestamps[i])
                    else:
                        s.append(threshold)
                s.append(min(m._future_timestamp - t, threshold))
                snapshot.append(s)
        
        if key not in meta:
            meta[key] = Meta(key, size, t, next_t)
        else:
            meta[key].update(t, next_t)


0it [00:00, ?it/s]
12553it [00:00, 125525.03it/s]
24979it [00:00, 125142.77it/s]
37472it [00:00, 125078.09it/s]
50325it [00:00, 126092.48it/s]
63227it [00:00, 126955.12it/s]
75766it [00:00, 126480.53it/s]
88392it [00:00, 126413.33it/s]
101083it [00:00, 126560.69it/s]
113565it [00:00, 126032.35it/s]
126012it [00:01, 125556.14it/s]
138580it [00:01, 125593.22it/s]
151071it [00:01, 125385.88it/s]
163437it [00:01, 123811.38it/s]
175907it [00:01, 124075.87it/s]
188234it [00:01, 123554.11it/s]
200533it [00:01, 123375.43it/s]
212876it [00:01, 123390.51it/s]
225214it [00:01, 123384.74it/s]
237555it [00:01, 123392.26it/s]
249881it [00:02, 123269.65it/s]
262325it [00:02, 123617.77it/s]
274687it [00:02, 123618.30it/s]
287065it [00:02, 123665.92it/s]
299441it [00:02, 123691.42it/s]
311860it [00:02, 123839.64it/s]
324346it [00:02, 124141.73it/s]
336760it [00:02, 124131.75it/s]
349173it [00:02, 123287.44it/s]
361503it [00:02, 119604.64it/s]
373693it [00:03, 120282.46it/s]
385874it [00:03, 120733.98i

In [9]:
len(snapshot)

8127766

In [10]:
snapshot[-10:]

[[4, 759, 10000000, 10000000, 10000000, 10000000],
 [4, 724, 10000000, 10000000, 10000000, 10000000],
 [4, 722, 10000000, 10000000, 10000000, 10000000],
 [4, 613, 10000000, 10000000, 10000000, 10000000],
 [4, 591, 10000000, 10000000, 10000000, 10000000],
 [4, 532, 10000000, 10000000, 10000000, 10000000],
 [4, 514, 10000000, 10000000, 10000000, 10000000],
 [4, 350, 10000000, 10000000, 10000000, 10000000],
 [4, 274, 10000000, 10000000, 10000000, 10000000],
 [4, 208, 10000000, 10000000, 10000000, 10000000]]

In [23]:
with open(f'{common.WEBCACHESIM_ROOT}/snapshot.log', 'w') as f:
    for s in snapshot:
        f.write(' '.join([str(i) for i in s]))
        f.write('\n')

In [22]:
' '.join([str(i) for i in snapshot[0]])

'1 723 730 806 870 475'

In [14]:
! head snapshot.log

head: cannot open 'snapshot.log' for reading: No such file or directory


In [24]:
fields = ['type', 'i1', 'i2', 'i3', 'i4', 'i0']
df_global = pd.read_csv(f'{common.WEBCACHESIM_ROOT}/snapshot.log', delimiter=' ', names=fields, nrows=100, skiprows=0)